# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 806ms/step - loss: 0.6628 - accuracy: 0.5949 - val_loss: 0.6887 - val_accuracy: 0.4585
Epoch 2/10
26/26 [==============================] - 20s 773ms/step - loss: 0.6159 - accuracy: 0.6655 - val_loss: 0.5660 - val_accuracy: 0.7854
Epoch 3/10
26/26 [==============================] - 20s 761ms/step - loss: 0.5633 - accuracy: 0.7324 - val_loss: 0.4975 - val_accuracy: 0.8439
Epoch 4/10
26/26 [==============================] - 20s 754ms/step - loss: 0.5440 - accuracy: 0.7543 - val_loss: 0.4621 - val_accuracy: 0.8634
Epoch 5/10
26/26 [==============================] - 20s 758ms/step - loss: 0.4602 - accuracy: 0.8041 - val_loss: 0.6970 - val_accuracy: 0.5805
Epoch 6/10
26/26 [==============================] - 20s 757ms/step - loss: 0.3906 - accuracy: 0.8467 - val_loss: 0.3223 - val_accuracy: 0.8878
Epoch 7/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3599 - accuracy: 0.8589 - val_loss: 0.4220 - val_accuracy: 0.8146

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 20s - loss: 0.6763 - accuracy: 0.5791 - val_loss: 0.6843 - val_accuracy: 0.4390
Epoch 2/5

Epoch 00002: saving model to weights.02-0.67.h5
26/26 - 20s - loss: 0.6532 - accuracy: 0.6521 - val_loss: 0.6674 - val_accuracy: 0.4585
Epoch 3/5

Epoch 00003: saving model to weights.03-0.60.h5
26/26 - 20s - loss: 0.6197 - accuracy: 0.7299 - val_loss: 0.5958 - val_accuracy: 0.8049
Epoch 4/5

Epoch 00004: saving model to weights.04-0.53.h5
26/26 - 20s - loss: 0.5775 - accuracy: 0.7652 - val_loss: 0.5339 - val_accuracy: 0.8000
Epoch 5/5

Epoch 00005: saving model to weights.05-0.67.h5
26/26 - 20s - loss: 0.5254 - accuracy: 0.7725 - val_loss: 0.6675 - val_accuracy: 0.5024


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6851 - accuracy: 0.5365 - val_loss: 0.6881 - val_accuracy: 0.4488


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 22s - loss: 0.6719 - accuracy: 0.5852 - val_loss: 0.6641 - val_accuracy: 0.5317
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6294 - accuracy: 0.7141 - val_loss: 0.7617 - val_accuracy: 0.4341


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6670 - accuracy: 0.6119 - val_loss: 0.7000 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 20s - loss: 0.6251 - accuracy: 0.6679 - val_loss: 0.5997 - val_accuracy: 0.7561
Epoch 3/50
26/26 - 20s - loss: 0.5491 - accuracy: 0.7798 - val_loss: 0.5244 - val_accuracy: 0.7659
Epoch 4/50
26/26 - 20s - loss: 0.4701 - accuracy: 0.7944 - val_loss: 0.5007 - val_accuracy: 0.7854
Epoch 5/50
26/26 - 20s - loss: 0.4403 - accuracy: 0.8005 - val_loss: 0.8161 - val_accuracy: 0.4976
Epoch 6/50
26/26 - 20s - loss: 0.4195 - accuracy: 0.8224 - val_loss: 0.3753 - val_accuracy: 0.8732
Epoch 7/50
26/26 - 20s - loss: 0.3317 - accuracy: 0.8856 - val_loss: 0.2621 - val_accuracy: 0.9220
Epoch 8/50
26/26 - 20s - loss: 0.2618 - accuracy: 0.9173 - val_loss: 0.1985 - val_accuracy: 0.9659
Epoch 9/50
26/26 - 20s - loss: 0.2243 - accuracy: 0.9246 - val_loss: 0.1637 - val_accuracy: 0.9756
Epoch 10/50
26/26 - 20s - loss: 0.1596 - accuracy: 0.9623 - val_loss: 0.1249 - val_accuracy: 0.9854
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 782ms/step - loss: 0.6826 - accuracy: 0.5864 - val_loss: 0.6751 - val_accuracy: 0.5707
Epoch 2/5
26/26 [==============================] - 20s 772ms/step - loss: 0.6301 - accuracy: 0.6521 - val_loss: 0.5997 - val_accuracy: 0.7073
Epoch 3/5
26/26 [==============================] - 20s 769ms/step - loss: 0.5526 - accuracy: 0.7530 - val_loss: 0.5223 - val_accuracy: 0.7610
Epoch 4/5
26/26 [==============================] - 20s 754ms/step - loss: 0.5165 - accuracy: 0.7616 - val_loss: 0.4667 - val_accuracy: 0.7707
Epoch 5/5
26/26 [==============================] - 20s 773ms/step - loss: 0.4399 - accuracy: 0.7932 - val_loss: 0.5322 - val_accuracy: 0.7317


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.586375,0.680500,0.570732,0.675133
1,1,0.652068,0.629359,0.707317,0.599662
2,2,0.753041,0.552599,0.760976,0.522296
3,3,0.761557,0.516600,0.770732,0.466727
4,4,0.793187,0.439709,0.731707,0.532195


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 30s 1s/step - loss: 0.6862 - accuracy: 0.5414 - val_loss: 0.6913 - val_accuracy: 0.4390

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 788ms/step - loss: 0.6666 - accuracy: 0.5511 - val_loss: 0.6752 - val_accuracy: 0.5366

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 766ms/step - loss: 0.6569 - accuracy: 0.6046 - val_loss: 0.6714 - val_accuracy: 0.5366

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 765ms/step - loss: 0.6518 - accuracy: 0.6119 - val_loss: 0.6692 - val_accuracy: 0.5415

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6493

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 835ms/step - loss: 0.6667 - accuracy: 0.5888 - val_loss: 0.6501 - val_accuracy: 0.6780
Epoch 2/50
26/26 [==============================] - 21s 814ms/step - loss: 0.6405 - accuracy: 0.6472 - val_loss: 0.6131 - val_accuracy: 0.7756
Epoch 3/50
26/26 [==============================] - 21s 789ms/step - loss: 0.6040 - accuracy: 0.6995 - val_loss: 0.6008 - val_accuracy: 0.5902
Epoch 4/50
25/26 [===========================>..] - ETA: 0s - loss: 0.5716 - accuracy: 0.7088
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 21s 808ms/step - loss: 0.5718 - accuracy: 0.7092 - val_loss: 0.6651 - val_accuracy: 0.5512
Epoch 5/50
26/26 [==============================] - 21s 792ms/step - loss: 0.5178 - accuracy: 0.7871 - val_loss: 0.5123 - val_accuracy: 0.8244
Epoch 6/50
26/26 [==============================] - 21s 812ms/step - loss: 0.4938 - accuracy: 0.8066 - val_loss: 0.4947 - 

In [ ]:
%tensorboard --logdir log_dir